# Event Bank

The EventBank class is used to interact with a local directory of event files.  The `get_events` method of Event Bank are compatible with the `get_events` method of the FDSN client in obspy. Additionally there are several useful features for managing events on disk.  


## Quickstart

In [ ]:
import obspy
import obsplus

In [ ]:
%%capture
# make sure the dataset is downloaded and supress output.
obsplus.load_dataset('crandall_test')

In [ ]:
# copy the Crandall dataset to a temporary directory
crandall = obsplus.copy_dataset('crandall_test')

# path to directory where events are stored
event_path = crandall.event_path

# init an EventBank instance
bank = obsplus.EventBank(event_path)

# ensure index is up-to-date
bank.update_index() 

### Accessing the index 
The index can be accessed directly to get a summary of the events contained in the archive. Depending on the task, it may be more natural to work with the index dataframe rather than the obspy catalog objects directly.

In [ ]:
index = bank.read_index()
index

The index contains the following columns:

In [ ]:
print(index.columns)

### Get events
The `EventBank` can be used to get obspy event objects based on query parameters.

In [ ]:
catalog = bank.get_events(minmagnitude=2)

In [ ]:
print(catalog)

## Put events
Events can be saved to disk using the `put_events` method. If an event with the same resource_id already exists in the bank it will be overwritten.

In [ ]:
print(f'The bank has {len(bank.read_index())} events before put_events call.')

In [ ]:
bank.put_events(obspy.read_events())
print(f'The bank has {len(bank.read_index())} events after the put_events call.')

## Organizing event directories
`EventBank` can also be used to (re)organize event directories. The events are saved in the following structure by default: `{year/month/day/year-month-dayThour-minute-second-short_id.xml}` (where "short_id" means the last 5 characters of the event id). 

The structure of the event directories can be reorganized to `{year/month/short_id.xml}`.  The following code demonstrates the process used in changing the event directory structure.



In [ ]:
from pathlib import Path

import tempfile

temp_dir = Path(tempfile.mkdtemp())

kwargs = dict(
    path_structure="{year}/{month}",
    name_structure="{event_id_short}",
)

In [ ]:
print(bank.get_events())

In [ ]:
bank2 = obsplus.EventBank(temp_dir, **kwargs)
bank2.put_events(bank)
print(bank2.read_index()['path'])

## Notes
Unlike the [WaveBank](wavebank.pynb), which uses HDF5 to index waveforms, EventBank uses a [SQLite](https://www.sqlite.org/index.html) which is more suitable for frequent updates and [CRUD](https://en.wikipedia.org/wiki/Create,_read,_update_and_delete) usage patterns.